In [1]:
from hello.pid.lvpid import PIDController
from hello.pid.delay import seconds, minutes, hours, days, m2s, s2m, h2s, DelaySink, DelayBuffer
import numpy as np

from hello.pid.gas_process import GasController, HeadspaceProcess

In [49]:
# Henry's law
def calc_hcp(T):
    C = 2400 
    T0 = 298
    hcp0 = 3.4e-2
    return hcp0*np.exp(C*(1/T - 1/T0))
hcp = calc_hcp(T=273.15 + 37)
hcp

0.024800416496804159

In [60]:
TOTAL_RVOLUME = {
    3: 4,
    15: 17,
    80: 90,
}

def qf(a,b,c):
    return (-b+np.sqrt(b**2-4*a*c))/(2*a)

def ph_to_co2(ph, hco3, hcp=hcp):
    h = 10**(-ph)
    w = 1e-7
    ka = 4.45e-7
    bc = hco3 + h - w
    co2 = bc * h / ka
    return co2 / hcp

def general_ph(c1, c2, c3, ka):
    a = 1
    b = c2+c3+ka
    c = c2*c3-ka*c1
    x=qf(a,b,c)
    return -np.log10(c2+x)

def cpH(co2, bicarb, hcp=hcp):
    # co2 as fraction of atmosphere
    c1 = co2 * hcp # henry's law
    c2 = 1e-7
    c3 = bicarb
    ka = 4.45e-7  # pka 6.35
    a = 1
    b = c2+c3+ka
    c = c2*c3-ka*c1
    x=qf(a,b,c)
    return -np.log10(c2+x)

class pHProcess():
    """ pH Process
    Uses cascaded calculation model,
    First using HeadspaceProcess to estimate
    headspace gas concentrations, and using
    the result to calculate change in PV. 
    """
    
    # First order constant
    default_k = (-0.19053+-0.17422)/2
    
    # Consumption rate
    default_c = 0
    
    # volume fraction of CO2, N2, O2 in air
    # note the remaining 1% is trace gasses
    AIR_CNO = (0.0004, 0.7809, 0.2095)
    
    def __init__(self, main_gas=1, initial_pv=7, initial_cno=AIR_CNO, reactor_size=80, volume=55, delay=0,
                 bicarb=2.02, temp=37):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        :param bicarb: bicarbonate concentration in g/L
        """
        self.sink = DelaySink(delay, initial_cno[2])
        self.delay = self.sink.delay
        
        self.hcp = calc_hcp(temp+273.15)
        self.k = 0
        self.k_mult = 0
        self._k = 0
        self.c = 0
        self.dc = 0
        self.d2c = 0
        self.set_values(k=self.default_k, k_mult=1, c=self.default_c, dc=0, d2c=0)
        self._bicarb = bicarb / 84  # moles / L
        self._co2 = ph_to_co2(initial_pv, self._bicarb, self.hcp)
        if self._co2 > 1:
            self._co2 = 1
        if self._co2 < 0:
            self._co2 = 0
        
        self._reactor_size = reactor_size
        self._volume = volume
        self.main_gas = main_gas
    
        hs_volume = TOTAL_RVOLUME[reactor_size] - volume
        c, n, o = initial_cno
        self.hp = HeadspaceProcess(hs_volume, c, n, o)
        
    @property
    def volume(self): 
        return self._volume
    
    def set_values(self, **kw):
        if any(not hasattr(self, a) for a in kw):
            raise KeyError(next(set(kw) - set(dir(self))))
            
        if kw.get('k') is not None or kw.get('k_mult') is not None:
            if kw.get('k') is not None:
                self.k = kw['k'] / 3600
            
            if kw.get('k_mult') is not None:
                self.k_mult = kw['k_mult']
                
            self._k = self.k_mult * self.k
    
        if kw.get('c') is not None:
            self.c = kw['c'] / 3600
        if kw.get('dc') is not None:
            self.dc = kw['dc'] / 3600 / 3600
        if kw.get('d2c') is not None:
            self.d2c = kw['d2c'] / 3600 / 3600 / 3600
            
        if kw.get('delay') is not None:
            self.sink.set_delay(kw['delay'])
    
    @volume.setter
    def volume(self, v):
        self._volume = v
        self.hp.vol = TOTAL_RVOLUME[self._reactor_size] - v

    def step(self, co2_req, n2_req, o2_req, air_req):
        
        # The convention of having pv as a parameter to the step 
        # function to minimize the class statefulness has to be 
        # broken here to track the concentration of [CO2] accurately. 
        # Because pH calculations are approximations using equilibrium
        # equations and involve very small numbers, small errors can
        # propagate to significant errors over time. 
        
        # [CO2] is tracked internally as the expected atmospheric 
        # concentration for the system's current PV. PV itself is not
        # stored internally - it is calculated on demand based on 
        # [CO2], assuming the system is at equilibrium (i.e., that
        # the equilibrium reaction happens much faster than [CO2]
        # changes).
        
        # This appears to corroborate with data collected on 3/14/17.
        
        self.hp.calc_gas(self.main_gas, co2_req, n2_req, o2_req, air_req)
        co2_hs = self.hp.co2A
        co2 = self._co2
        dco2 = self._k * (co2 - co2_hs)

        co2 += dco2
        
        self.dc += self.d2c
        self.c += self.dc
        co2 += self.c
        
        if co2 < 0:
            co2 = 0
        elif co2 > 1:
            co2 = 1
        self._co2 = co2
        pv = cpH(co2, self._bicarb, self.hcp)
        return pv 

In [61]:
import json
from pysrc.snippets import OptionCategory

class PIDOps(OptionCategory):
    p = 5
    i = 5
    d = 0
    amax = 100
    amin = 0
    alpha = 1
    beta = 1
    linearity = 1
    gamma = 0
    deadband = 0
    man_request = 0
    mode = 0
    
class MFCOps(OptionCategory):
    co2_max = 1
    o2_max = 2
    n2_max = 10
    air_max = 10
    
class PlotOps(OptionCategory):
    xscale = 'auto'
    xmin = 0
    xmax = 72
    xscale_factor = 3600

class SimOps(OptionCategory):
    
    co2_pid = PIDOps()
    co2_pid.p = -200
    co2_pid.i = 2
    co2_pid.d = 0
    co2_pid.amax = 100
    co2_pid.amin = 0
    co2_pid.beta = 1
    co2_pid.linearity = 1
    co2_pid.alpha = 1
    
    base_pid = PIDOps()
    base_pid.p = 20
    base_pid.i = 2

    mfcs = MFCOps()
    mfcs.co2_max = 1
    mfcs.o2_max = 10
    mfcs.n2_max = 10
    mfcs.air_max = 10
    
    plots = PlotOps()
    plots.xscale = 'auto'
    plots.xmin = 0
    plots.xmax = 72
    plots.xscale_factor = 3600
    
    delay = 0
    end = 10000
    initial_actual_cno = pHProcess.AIR_CNO
    initial_request_cno = (0.07, 0, 0)
    initial_request_base = 0
    initial_pv = 90
    set_point = 40
    set_point_deadband = 1
    k_mult = 1.1
    k = None
    c = None
    dc = 0
    d2c = 0
    mode = "m2a"
    main_gas = 1.0
    reactor_size = 80
    reactor_volume = reactor_size * 55/80
    time_unit = hours
    max_iters = 3 * days
    bicarb = 3.7
    temp = 37


In [62]:
ops = SimOps()

ops.co2_pid.p = -200
ops.co2_pid.i = 2
ops.co2_pid.d = 0
ops.co2_pid.amax = 30
ops.co2_pid.amin = 0
ops.co2_pid.alpha = -1
ops.co2_pid.linearity = 1
ops.co2_pid.beta = 0
ops.co2_pid.gamma = 0
ops.co2_pid.deadband = 0
ops.co2_pid.man_request = 0
ops.co2_pid.mode = 0

ops.base_pid.p = 20
ops.base_pid.i = 2
ops.base_pid.d = 0
ops.base_pid.amax = 20
ops.base_pid.amin = 0
ops.base_pid.alpha = -1
ops.base_pid.beta = 1
ops.base_pid.linearity = 1
ops.base_pid.gamma = 0
ops.base_pid.deadband = 0
ops.base_pid.man_request = 0
ops.base_pid.mode = 0

ops.mfcs.co2_max = 1
ops.mfcs.o2_max = 2
ops.mfcs.n2_max = 10
ops.mfcs.air_max = 10

ops.delay = 0
ops.end = 10*hours
ops.initial_actual_cno = pHProcess.AIR_CNO
ops.initial_request_cno = (0.07, 0, 0)
ops.initial_pv = 7
ops.set_point = 7.21
ops.set_point_deadband = 0.02
ops.k = pHProcess.default_k
ops.k_mult = 1.0
ops.c = 0
ops.dc = 0
ops.d2c = 0
ops.mode = "o2a"
ops.main_gas = 1.0
ops.reactor_size = 80
ops.reactor_volume = 55
ops.time_unit = hours
ops.max_iters = 7 * days
ops.bicarb = 3.7
ops.temp = 37

In [63]:
def _mk_pid(pidops, pv, sp, req, mode):
    c = PIDController(pgain=pidops.p,              itime=pidops.i,
                            dtime=pidops.d,             auto_max=pidops.amax,
                            auto_min=pidops.amin,       
                            beta=pidops.beta,
                            linearity=pidops.linearity, alpha=pidops.alpha,
                            deadband=pidops.deadband,   sp_high=100, sp_low=0,
                            gamma=pidops.gamma,         man_request=pidops.man_request,
                            mode=pidops.mode)
    if mode == "o2a":
        c.off_to_auto(pv, sp)
    elif mode == "m2a":
        c.man_to_auto(pv, sp, req)
    elif mode == "a2a":
        c.man_to_auto(pv, pv, req)
    else:
        raise ValueError(mode)
    return c

def _setpid(ob, state, name, value):
    pid = state[ob]
    if name == "pgain":
        pid.pgain = value
    elif name == "itime":
        pid.itime = m2s(value)
    elif name == "dtime":
        pid.dtime = m2s(value)
    elif name == "beta":
        pid.b = value
    elif name == "mode":
        sp = state['sp']
        if ob[:2] == "n2":
            sp += state['db']
        elif ob[:2] == 'o2':
            sp -= state['db']
        else:
            print("BAD PID")
        pid.set_mode(value, state['pv'], sp)
    elif name == "man":
        pid.man_request = int(value)
    else:
        print("WARNING: Invalid attribute for %r: %r" % (ob, name, name == "man"))

def _update_value(ops, state, proc, ob, name, value):
    if ob == "process":
        if name == "delay":
            value *= 60
        proc.set_values(**{name:value})
        return
    elif ob in ("co2_pid", "base_pid"):
        _setpid(ob, state, name, value)
        return
    print("WARNING: Invalid attribute or object: %r %r" % (ob, name))
    
    
def ph_sim_coroutine(ops, state, xq, pvq, cq,
                     co2q, n2q, o2q, aq, baseq,
                     co2ukq, co2upq, co2uiq, co2udq,
                     baseukq, baseupq, baseuiq, baseudq, 
                     co2slnq, co2hsq):
    
    """ This is the longest function in the history of man. 
    Its long becaues there are a *lot* of values to use and unpack for the inner PID loop,
    including code needed to synchronize with UI updates.
    Yay. 
    """
    
    _hcalc = hcalc
    
    co2_req, n2_req, o2_req = ops.initial_request_cno
    base_req = ops.initial_request_base
    air_req = max(1-co2_req - n2_req - o2_req, 0)
    pv = ops.initial_pv
    sp = ops.set_point
    mode = ops.mode
    db = ops.set_point_deadband
    
    co2_pid = _mk_pid(ops.co2_pid, _hcalc(ops.initial_pv), _hcalc(sp+db), co2_req*100, ops.mode)
    base_pid = _mk_pid(ops.base_pid, _hcalc(ops.initial_pv), _hcalc(sp-db), base_req*100, ops.mode)

    delay = ops.delay
    mg = ops.main_gas
    co2a, n2a, o2a = ops.initial_actual_cno
    
    proc = pHProcess(mg, pv,     (co2a, n2a, o2a), 
                     ops.reactor_size, ops.reactor_volume, 
                     ops.delay, ops.bicarb, ops.temp)
    
    proc.set_values(k=ops.k, k_mult=ops.k_mult, c=ops.c, dc=ops.dc, d2c=ops.d2c)
    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    
    t = 0
    time_unit = ops.time_unit
    mi = ops.max_iters
    msg = None
    
    while True:

        cmd, arg = yield msg
        msg = None
        
        state['c'] = proc.c * 3600
        state['dc'] = proc.dc * 3600 * 3600
        state['d2c'] = proc.d2c * 3600 * 3600 * 3600
        state['k'] = proc.k
        state['sp'] = sp
        state['db'] = db
        state['pv'] = pv
        state['mg'] = mg
        state['t'] = t
        state['co2_req'] = co2_req
        state['n2_req'] = n2_req
        state['o2_req'] = o2_req
        state['air_req'] = air_req
        state['co2_pid'] = co2_pid
        state['base_pid'] = base_pid
        state['base_req'] = base_req
        state['ph_process'] = proc
        
        if cmd == "SIM_ITERS":
            
            iters = arg
            if iters < 0:
                continue
            elif iters > mi:
                iters = mi

            # One approximation made in this simulation
            # is that no values, including oxygen consumption
            # (c), change during the inner iteration period 
            # outside of modification by the process or PID
            # classes themselves. 

            while iters > 0:

                t += 1
                iters -= 1
                co2_sp = _hcalc(sp+db)
                base_sp = _hcalc(sp-db)
                h_pv = _hcalc(pv)
                co2_req = co2_pid.step(h_pv, co2_sp) / 100
                base_req = base_pid.step(h_pv, base_sp) / 100
                co2_req, n2_req, o2_req, air_req = ctrl.request(mg, co2_req, n2_req, o2_req)
                pv = proc.step(co2_req, n2_req, o2_req, air_req)

                xq.put(t/time_unit)
                pvq.put(pv)
                cq.put(proc.c * 3600)

                co2q.put(co2_req)
                n2q.put(n2_req)
                o2q.put(o2_req)
                aq.put(air_req)
                baseq.put(base_req)
                
                co2slnq.put(proc._co2)
                co2hsq.put(proc.hp.co2A)

                co2ukq.put(co2_pid.Uk)
                co2upq.put(co2_pid.Up)
                co2uiq.put(co2_pid.Ui)
                co2udq.put(co2_pid.Ud)

                baseukq.put(base_pid.Uk)
                baseupq.put(base_pid.Up)
                baseuiq.put(base_pid.Ui)
                baseudq.put(base_pid.Ud)

        elif cmd == "SET_TIME":
            t = arg
        elif cmd == "UPDATE_VALUE":
            ob, name, value = arg
            rsp = _update_value(ops, state, proc, ob, name, value)
        elif cmd == "MODIFY_STATE":
            name, value = arg
            if name == "sp":
                sp = state['sp'] = value
            elif name == "pv":
                pv = state['pv'] = value
            elif name == "main_gas":
                mg = state['mg'] = value
            else:
                print("WARNING: Unrecognized name: %r" % name)
        else:
            raise ValueError(cmd)


In [64]:
def ph_sim(ops):
    # standalone sim function compatible with the do_sim_coroutine
    # function used for interactive use
    global state
    state = {}
    
    class MyList(list):
        put = list.append
    
    xq = MyList()
    pvq = MyList()
    cq = MyList()
    co2q = MyList()
    n2q = MyList()
    o2q = MyList()
    aq = MyList()
    baseq = MyList()
    co2ukq = MyList()
    co2upq = MyList()
    co2uiq = MyList()
    co2udq = MyList()
    baseukq = MyList()
    baseupq = MyList()
    baseuiq = MyList()
    baseudq = MyList()
    co2slnq = MyList()
    co2hsq = MyList()
    
    coro = ph_sim_coroutine(ops, state, xq, pvq, cq,
                     co2q, n2q, o2q, aq, baseq,
                     co2ukq, co2upq, co2uiq, co2udq,
                     baseukq, baseupq, baseuiq, baseudq,
                     co2slnq, co2hsq)
    
    next(coro)
    coro.send(("SIM_ITERS", ops.end))
    
    data = list(zip(xq, pvq, cq,
                     co2q, n2q, o2q, aq, baseq))
    data2 = list(zip(co2ukq, co2upq, co2uiq, co2udq))
    data3 = list(zip(baseukq, baseupq, baseuiq, baseudq))
    data4 = co2slnq, co2hsq
    return data, data2, data3, data4
    

In [65]:
# O2 needs an extra padding space because the negative sign
# for N2 pgain doesn't count, i guesss
def fmt_float(f, decimals=3):
    if not f:
        return "0"
    f = str(f)
    if "." not in f:
        return f
    else:
        f = f.strip("0")
        if f[-1] == ".":
            f = f[:-1]
        if f[0] == ".":
            f = "0" + f
        if "." in f:
            if decimals > 0:
                f = f[:f.find(".")+decimals+1]
            else:
                f = f[:f.find(".")]
        return f
    
def get_text(ops):

    text = \
"""
CO2: P=%.4f  I=%.4f  Initial PV: %3d  k: %.4f/hr (%sx Est.)
                   Set Point:  %3d  c: %s%%/hr

 """ % (ops.co2_pid.p, ops.co2_pid.i,ops.initial_pv, ops.k*ops.k_mult, fmt_float(ops.k_mult),
        ops.set_point, fmt_float(ops.c))
    text = text.strip()
    return text

In [10]:
def run(ops):
    global data, data2, text, data3, data4
    data, data2, data3, data4 = ph_sim(ops)
    text = get_text(ops)
    plot(ops)

In [11]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, FormatStrFormatter
%matplotlib

def plot(ops):    
    print("Parsing Data")
    global x, pv, mg, cs, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a, uk, up, ui, ud
    x, pv, cs, co2_req, n2_req, o2_req, air_req, base_req = list(zip(*data))
    uk, up, ui, ud = list(zip(*data2))
    
    print("Closing Plot")
    plt.close()
    print("Plotting Data")

    step = 100
    xs = x[::step]
    xs = np.array(xs) / ops.plots.xscale_factor * ops.time_unit
    fig = plt.figure()
    ax1 = plt.subplot(311)
    ax2 = plt.subplot(312)
    ax3 = plt.subplot(313)

    pvs = np.array(pv[::step])
    ax1.plot(xs, pvs, "blue", ls="-", label="PV")
    ax1.axhline(y=sp, ls="--", color="black")

    ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
    ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
    ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
    ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
    fm1 = FuncFormatter(lambda y, _: "%.2f"%y)
    fm2 = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))

    ax3.plot(xs, uk[::step], "blue", ls="-", label="Uk")
    ax3.plot(xs, up[::step], "red", ls="-", label="Up")
    ax3.plot(xs, ui[::step], "green", ls="-", label="Ui")
    ax3.plot(xs, ud[::step], "purple", ls="-", label="Ud")

    ax1.yaxis.set_major_formatter(fm1)
    ax2.yaxis.set_major_formatter(fm2)
    ax2.set_ylim((0, 1.1))
    
    m, ma = ax1.get_ylim()
    nmin = np.min(pvs)
    nmax = np.max(pvs)
    m = nmin - 1
    ma = nmax + 1
    ax1.set_ylim((m, ma))

    for a in ax1, ax2, ax3: 
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.9, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.grid()
        if ops.plots.xscale == 'man':
            a.set_xlim((ops.plots.xmin, ops.plots.xmax))
    
    fig.text(0.15, 0.95, get_text(ops), transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
    wm=plt.get_current_fig_manager()
    wm.window.attributes('-topmost', 1)
    wm.window.attributes('-topmost', 0)
    # h = wm.window.winfo_height()
    # w = wm.window.winfo_width()
    wm.window.geometry("%sx%s+%s+%s"%(700,720,50, 20))
    
def run_int(ops):
    global data, data2, text, data3, data4
    data, data2, data3, data4 = ph_sim(ops)
    text = get_text(ops)
    plot_int(ops)
    
def run_int2(ops):
    global data, data2, text, data3, data4
    data, data2, data3, data4 = ph_sim(ops)
    text = get_text(ops)
    plot_int2(ops)

Using matplotlib backend: TkAgg


In [12]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, FormatStrFormatter
%matplotlib

def plot_int(ops):    
    print("Parsing Data")
    global x, pv, mg, cs, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a, uk, up, ui, ud
    global ax1, ax2, ax3, fig
    x, pv, cs, co2_req, n2_req, o2_req, air_req, base_req = list(zip(*data))
    uk, up, ui, ud = list(zip(*data2))
    co2sln, co2hs = data4
    print("Closing Plot")
    plt.close()
    print("Plotting Data")

    step = 100
    xs = x[::step]
    xs = np.array(xs) / ops.plots.xscale_factor * ops.time_unit
    fig = plt.figure()
    ax1 = plt.subplot(311)
    ax2 = plt.subplot(312)
    ax3 = plt.subplot(313)

    pvs = np.array(pv[::step])
    ax1.plot(xs, pvs, "blue", ls="-", label="PV")
    #ax1.axhline(y=sp, ls="--", color="black")

    ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
    ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
    ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
    ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
    fm1 = FuncFormatter(lambda y, _: "%.2f"%y)
    fm2 = FuncFormatter(lambda y, _: "%.1f%%"%(y*100))

    ax3.plot(xs, co2sln[::step], "blue", ls="-", label="co2sln")
    ax3.plot(xs, co2hs[::step], "red", ls="-", label="co2hs")
    #ax3.plot(xs, np.array(co2hs)[::step]-np.array(co2sln)[::step], "green", label="diff")
    
    ax1.yaxis.set_major_formatter(fm1)
    ax2.yaxis.set_major_formatter(fm2)
    ax3.yaxis.set_major_formatter(fm2)
    ax2.set_ylim((0, 1.1))
    
    m, ma = ax1.get_ylim()
    nmin = np.min(pvs)
    nmax = np.max(pvs)
    m = nmin - 1
    ma = nmax + 1
    #ax1.set_ylim((m, ma))

    for a in ax1, ax2, ax3: 
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.9, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.grid()
        if ops.plots.xscale == 'man':
            a.set_xlim((ops.plots.xmin, ops.plots.xmax))
    
    fig.text(0.15, 0.95, get_text(ops), transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
    wm=plt.get_current_fig_manager()
    wm.window.attributes('-topmost', 1)
    wm.window.attributes('-topmost', 0)
    # h = wm.window.winfo_height()
    # w = wm.window.winfo_width()
    wm.window.geometry("%sx%s+%s+%s"%(700,720,50, 20))

Using matplotlib backend: TkAgg


In [13]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, FormatStrFormatter
from hello.pid.picker import mk_picker
%matplotlib

def np_unpack(data):
    return [np.array(a) for a in list(zip(*data))]

def plot_int2(ops):    
    print("Parsing Data")
    global x, pv, mg, cs, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a, uk, up, ui, ud
    global ax1, ax2, ax3, fig
    x, pv, cs, co2_req, n2_req, o2_req, air_req, base_req = np_unpack(data)
    uk, up, ui, ud = list(zip(*data2))
    co2sln, co2hs = data4
    print("Closing Plot")
    plt.close()
    print("Plotting Data")

    step = 100
    xs = x[::step]
    xs = np.array(xs) / ops.plots.xscale_factor * ops.time_unit
    fig = plt.figure()
    ax1 = plt.subplot(211)
    ax2 = plt.subplot(212)
    
    co2hs = np.array(co2hs)
    eph = cpH(co2hs, ops.bicarb/84)

    pvs = np.array(pv[::step])
    ax1.plot(xs, pvs, "blue", ls="-", label="PV")
    ax1.plot(xs, eph[::step], "green", ls="-", label="EXP")
    
    fm1 = FuncFormatter(lambda y, _: "%.2f"%y)
    fm2 = FuncFormatter(lambda y, _: "%.1f%%"%(y*100))

    ax2.plot(xs, co2sln[::step], "blue", ls="-", label="co2sln")
    ax2.plot(xs, co2hs[::step], "red", ls="-", label="co2hs")
    #ax3.plot(xs, np.array(co2hs)[::step]-np.array(co2sln)[::step], "green", label="diff")
    
    ax1.yaxis.set_major_formatter(fm1)
    ax2.yaxis.set_major_formatter(fm2)
    
    m, ma = ax1.get_ylim()
    nmin = np.min(pvs)
    nmax = np.max(pvs)
    m = nmin - 1
    ma = nmax + 1
    #ax1.set_ylim((m, ma))

    for a in ax1, ax2: 
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.9, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.grid()
        if ops.plots.xscale == 'man':
            a.set_xlim((ops.plots.xmin, ops.plots.xmax))
        mk_picker(fig, a)
    
    fig.text(0.15, 0.95, get_text(ops), transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
    wm=plt.get_current_fig_manager()
    wm.window.attributes('-topmost', 1)
    wm.window.attributes('-topmost', 0)
    # h = wm.window.winfo_height()
    # w = wm.window.winfo_width()
    wm.window.geometry("%sx%s+%s+%s"%(700,720,50, 20))
    

Using matplotlib backend: TkAgg


In [18]:
def hcalc(v):
    return v

In [20]:
ops.end = 15 * hours


ops.co2_pid.p = -200
ops.co2_pid.i = 50
ops.co2_pid.amax = 25
ops.co2_pid.deadband = 0
ops.co2_pid.d = 0
ops.co2_pid.alpha = -1

ops.base_pid.p = 20
ops.base_pid.i = 2
ops.base_pid.amax = 100
ops.base_pid.deadband = 0
ops.base_pid.d = 0
ops.base_pid.alpha = -1



ops.c = 0
ops.d2c = 0.0

# k_mult = 1.25
ops.k_mult = 1
ops.k = pHProcess.default_k
ops.initial_pv = 7.5
ops.set_point = sp = 7.3
ops.initial_actual_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84),0,0)
ops.bicarb = 2.02
ops.initial_request_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84), 0, 0)
ops.mode = 'a2a'
ops.plots.xscale = 'auto'
ops.plots.xmin = 20
ops.plots.xmax = 30
ops.set_point_deadband = 0.02
ops.delay = 0


run_int2(ops)

from matplotlib.ticker import MultipleLocator
l = MultipleLocator(1)
for a in plt.gcf().axes:
    a.xaxis.set_major_locator(l)
ax1.axhline(y=sp, ls="--", color="black")
ax1.set_ylim(sp-0.2, sp+0.2)
x_settle = np.max(np.where(np.abs(pv-sp)>0.05))/3600
ax1.axvline(x=x_settle, color="black", ls="--")

Parsing Data
Closing Plot
Plotting Data


In [24]:
def plot3(x, pv, sp, label):
    ax.plot(x, pv, label=label)
    #ax.plot(x, pv-sp)
    h1, l1 = ax.get_legend_handles_labels()
    ax.legend(h1, l1, bbox_to_anchor=(0.99, 1.06), loc="upper left")
    try:
        settle_x = np.max(np.where(np.abs(pv-sp)>0.05))
        if settle_x == len(pv) - 1 and pv[-1] - sp > 0.05:
            raise ValueError()  # don't plot '+'
    except ValueError:
        pass
    else:
        ax.plot(x[settle_x], pv[settle_x], "+", mfc=None, mec='r', mew=2, ms=8)
    fig.canvas.flush_events()
    fig.canvas.draw()
    print("I: %d   Settle: %s" % (i, _hms(x[settle_x])))

In [25]:
def test3(ops):
    data1, data2, data3, data4 = ph_sim(ops)
    x, pv, _, co2_r, *_ = np_unpack(data1)
    return x, pv, co2_r

In [33]:
if not plt.get_fignums():
    fig = plt.figure()
    ax = fig.add_subplot(111)
    b = ax.get_position()
    ax.set_position([b.x0, b.y0, b.width*0.9, b.height])
else:
    ax.clear()

def _hms(hrs):
    h = int(hrs)
    min = (hrs-h) * 60
    m = int(min)
    sec = (min-m) *60
    s = int(sec)
    return "%d:%d" % (h,m)

def hcalc(v):
    return 10**(-v)*1e8
    

ops.end = 15*hours
ops.co2_pid.p = 15
ops.co2_pid.i = 50
ops.co2_pid.amax = 25
ops.co2_pid.deadband = 0
ops.co2_pid.d = 0
ops.co2_pid.alpha = -1
ops.co2_pid.beta = 0

ops.initial_pv = 7.5
ops.set_point = sp = 7.3
ops.initial_actual_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84),0,0)
ops.bicarb = 2.02
ops.initial_request_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84), 0, 0)

ax.axhline(y=sp+0.05, color="black", ls="--", linewidth=0.5)
ax.axhline(y=sp, color="black", ls="--")
ax.axhline(y=sp-0.05, color="black", ls="--", linewidth=0.5)
ax.set_ylim(sp-0.1, ops.initial_pv)


for i in range(20, 111, 20):
    ops.co2_pid.i = i
    x, pv, op= test3(ops)
    plot3(x, pv, ops.set_point, "I:%d"%i)
    
mk_picker(fig, ax)

I: 20   Settle: 12:35
I: 40   Settle: 5:35
I: 60   Settle: 1:58
I: 80   Settle: 2:23
I: 100   Settle: 2:49


In [34]:
def hcalc(v):
    return 10**(-v)*1e8

def plot4(x, pv, op, sp, label):
    ax1.plot(x, pv, label=label)
    ax2.plot(x, pv-sp, label=label)
    ax3.plot(x, op, label=label)
    for a in ax1, ax2, ax3:
        h1, l1 = a.get_legend_handles_labels()
        a.legend(h1, l1, bbox_to_anchor=(0.99, 1.06), loc="upper left")
    try:
        settle_x = np.max(np.where(np.abs(pv-sp)>0.05))
        if settle_x == len(pv) - 1 and pv[-1] - sp > 0.05:
            raise ValueError()  # don't plot '+'
    except ValueError:
        pass
    else:
        ax1.plot(x[settle_x], pv[settle_x], "+", mfc=None, mec='r', mew=2, ms=8)
    fig.canvas.flush_events()
    fig.canvas.draw()
    print("I: %d   Settle: %s" % (i, _hms(x[settle_x])))

if not plt.get_fignums():
    fig = plt.figure()
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)
    for a in ax1, ax2, ax3:
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.8, b.height])
else:
    ax1.clear()
    ax2.clear()
    ax3.clear()

for a in (ax1, ax2, ax3):
    a.grid()
ops.co2_pid.p = 15
ops.co2_pid.i = 80
ops.co2_pid.amax = 100
ops.bicarb = 2.02
data = {}
for ipv in 7.5, 7.3, 7.1:
    ops.initial_pv = ipv
    ops.set_point = sp = ipv - 0.2
    ops.initial_actual_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84),0,0)
    ops.initial_request_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84), 0, 0)
    x, pv, op = test3(ops)
    data[ipv] = x,pv
    plot4(x,pv, op, sp, "Initial: %.1f"%ipv)
#ax1.axhline(y=0, color="black", ls="--")

for a in (ax1, ax2, ax3):
    mk_picker(fig, a)

I: 100   Settle: 2:16
I: 100   Settle: 2:16
I: 100   Settle: 2:16


In [66]:
def test_the_thing(p,i,bicarb=2.02, temp=37):
    def plot4(x, pv, op, sp, label):
        ax1.plot(x, pv, label=label)
        ax2.plot(x, pv-sp, label=label)
        ax3.plot(x, op, label=label)
        for a in ax1, ax2, ax3:
            h1, l1 = a.get_legend_handles_labels()
            a.legend(h1, l1, bbox_to_anchor=(0.99, 1.06), loc="upper left")
        try:
            settle_x = np.max(np.where(np.abs(pv-sp)>0.05))
            if settle_x == len(pv) - 1 and pv[-1] - sp > 0.05:
                raise ValueError()  # don't plot '+'
        except ValueError:
            pass
        else:
            ax1.plot(x[settle_x], pv[settle_x], "+", mfc=None, mec='r', mew=2, ms=8)
        fig.canvas.flush_events()
        fig.canvas.draw()
        print("I: %d   Settle: %s" % (i, _hms(x[settle_x])))

    fig = plt.figure()
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)
    for a in ax1, ax2, ax3:
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.8, b.height])

    ax3.yaxis.set_major_formatter(FuncFormatter(lambda x, _: "%.2f%%"%(x*100)))
    for a in (ax1, ax2, ax3):
        a.grid()
        
    ops = SimOps()
    ops.co2_pid.p = p
    ops.co2_pid.i = i
    ops.co2_pid.amax = 100
    ops.bicarb = bicarb

    ops.co2_pid.d = 0
    ops.co2_pid.amin = 0
    ops.co2_pid.alpha = -1
    ops.co2_pid.linearity = 1
    ops.co2_pid.beta = 0
    ops.co2_pid.gamma = 0
    ops.co2_pid.deadband = 0
    ops.co2_pid.man_request = 0
    ops.co2_pid.mode = 0

    ops.base_pid.p = 20
    ops.base_pid.i = 2
    ops.base_pid.d = 0
    ops.base_pid.amax = 20
    ops.base_pid.amin = 0
    ops.base_pid.alpha = -1
    ops.base_pid.beta = 1
    ops.base_pid.linearity = 1
    ops.base_pid.gamma = 0
    ops.base_pid.deadband = 0
    ops.base_pid.man_request = 0
    ops.base_pid.mode = 0

    ops.mfcs.co2_max = 1
    ops.mfcs.o2_max = 2
    ops.mfcs.n2_max = 10
    ops.mfcs.air_max = 10

    ops.delay = 0
    ops.end = 15*hours
    ops.set_point_deadband = 0.02
    ops.k = pHProcess.default_k
    ops.k_mult = 1.0
    ops.c = 0
    ops.dc = 0
    ops.d2c = 0
    ops.mode = "a2a"
    ops.main_gas = 1.0
    ops.reactor_size = 80
    ops.reactor_volume = 55
    ops.time_unit = hours
    ops.max_iters = 7 * days
    ops.temp = temp
    
    hcp = calc_hcp(temp+273.15)
    
    data = {}
    for ipv in 7.7, 7.5, 7.3:
        ops.initial_pv = ipv
        ops.set_point = sp = ipv - 0.2
        ops.initial_actual_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84, hcp),0,0)
        ops.initial_request_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84, hcp), 0, 0)
        x, pv, op = test3(ops)
        data[ipv] = x,pv
        plot4(x,pv, op, sp, "Initial: %.1f"%ipv)
    #ax1.axhline(y=0, color="black", ls="--")

    for a in (ax1, ax2, ax3):
        mk_picker(fig, a)
    return locals().copy()


In [45]:
def test_inv_log_ph(bc=2.02):
    global hcalc
    def hcalc(v):
        return 10**(-v)*1e8
    l = test_the_thing(15, 80, bc)
    
    fig = l['fig']
    txt = "new_pv=10**(-pv)*1e8, hco3=%.2f (Error based on H+ gradient)"%bc
    fig.text(0.15, 0.95, txt, transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
test_inv_log_ph(2.02) 
test_inv_log_ph(3.7)

I: 80   Settle: 2:16
I: 80   Settle: 2:16
I: 80   Settle: 2:16
I: 80   Settle: 2:52
I: 80   Settle: 2:52
I: 80   Settle: 2:52


In [82]:
def test_ph_to_co2(bc, temp):
    global hcalc
    hcp = calc_hcp(temp+273.15)
    def hcalc(v):
        return ph_to_co2(v, bc/84, hcp)
    l = test_the_thing(500, 80, bc, temp)

    fig = l['fig']
    txt = "newpv = ph_to_co2(pv, hco3=%.2f), T=%dC (Error based on H+ gradient)" % (bc,temp)
    fig.text(0.15, 0.95, txt, transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
test_ph_to_co2(2.02, 37)  
test_ph_to_co2(3.7, 37)
test_ph_to_co2(2.02, 25)

I: 80   Settle: 2:33
I: 80   Settle: 2:33
I: 80   Settle: 2:33
I: 80   Settle: 2:33
I: 80   Settle: 2:33
I: 80   Settle: 2:33
I: 80   Settle: 2:33
I: 80   Settle: 2:33
I: 80   Settle: 2:33


In [85]:
def test_ph(bc):
    global hcalc
    def hcalc(v):
        return v
    l = test_the_thing(-200, 80, bc, 37)

    fig = l['fig']
    txt = "new_pv=pv, hco3=%.2f T=37(Error based on pH gradient)"%bc
    fig.text(0.15, 0.95, txt, transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
test_ph(2.02)  
test_ph(3.7)

I: 80   Settle: 1:51
I: 80   Settle: 2:5
I: 80   Settle: 2:26
I: 80   Settle: 2:11
I: 80   Settle: 2:34
I: 80   Settle: 3:4


In [86]:
plt.get_fignums()

[1, 2, 5, 6, 7, 8, 9]

In [87]:
args = [
    (1,2),
    #(3,4),
    [5,6,7],
    (8,9)
]
for arg in args:
    for i in arg:
        f = plt._pylab_helpers.Gcf.get_fig_manager(i).canvas.figure
        ax = f.axes[2]
        ax.set_ylim(0, .8)

In [ ]:
def plot5(x, pv, op, sp, label):
    hpv = hcalc(pv)
    ax1.plot(x, pv, label=label)
    ax2.plot(x, hcalc(sp)-hcalc(pv), label=label)
    #ax2.plot(x, hcalc(sp)-hcalc(pv), label=label)
    ax3.plot(x, op, label=label)
    for a in ax1, ax2, ax3:
        h1, l1 = a.get_legend_handles_labels()
        a.legend(h1, l1, bbox_to_anchor=(0.99, 1.06), loc="upper left")
    try:
        settle_x = np.max(np.where(np.abs(pv-sp)>0.05))
        if settle_x == len(pv) - 1 and pv[-1] - sp > 0.05:
            raise ValueError()  # don't plot '+'
    except ValueError:
        pass
    else:
        ax1.plot(x[settle_x], pv[settle_x], "+", mfc=None, mec='r', mew=2, ms=8)
    fig.canvas.flush_events()
    fig.canvas.draw()
    
    
def test5(ops):
    data1, data2, data3, data4 = ph_sim(ops)
    x, pv, _, co2r, *_ = np_unpack(data1)
    return x, pv, co2r

In [ ]:
if not plt.get_fignums():
    fig = plt.figure()
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(3,1,3)
    for a in ax1, ax2, ax3:
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.8, b.height])
        a.grid()
    
else:
    ax1.clear()
    ax2.clear()
    ax3.clear()
    for a in ax1, ax2, ax3:
        a.grid()
    
#ax1.set_ylim(7, 7.4)
ax1.yaxis.set_major_formatter(FormatStrFormatter("%.3f"))
ax3.yaxis.set_major_formatter(FuncFormatter(lambda x, _: "%.2f%%"%(x*100)))
ops.initial_pv = ipv = 7.3
ops.set_point = sp = ipv - 0.2

sp += 0.02
ax1.axhline(y=sp+0.05, color="black", ls="--", linewidth=0.5)
ax1.axhline(y=sp, color="black", ls="--")
ax1.axhline(y=sp-0.05, color="black", ls="--", linewidth=0.5)
sp -= 0.02

# ax2.axhline(y=hcalc(sp+0.05), color="black", ls="--", linewidth=0.5)
# ax2.axhline(y=hcalc(sp), color="black", ls="--")
# ax2.axhline(y=hcalc(sp-0.05), color="black", ls="--", linewidth=0.5)

ops.co2_pid.p = 15
ops.co2_pid.i = 0
ops.co2_pid.beta = 1
ops.bicarb = 2.02
data = {}
for i in (60, 80, 100):
    ops.co2_pid.i=i
    ops.initial_actual_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84),0,0)
    ops.initial_request_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84), 0, 0)
    x, pv, op = test5(ops)
    data[ipv] = x,pv
    plot5(x, pv, op, sp, "i:%d"%i)
for a in (ax1, ax2, ax3):
    mk_picker(fig, a)

In [ ]:
if not plt.get_fignums():
    fig = plt.figure()
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(3,1,3)
    for a in ax1, ax2, ax3:
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.8, b.height])
        a.grid()
    
else:
    ax1.clear()
    ax2.clear()
    ax3.clear()
    for a in ax1, ax2, ax3:
        a.grid()

p = 15
ops.co2_pid.p = p
ops.co2_pid.i = 80
ops.initial_actual_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84),0,0)
ops.initial_request_cno = (ph_to_co2(ops.initial_pv, ops.bicarb/84), 0, 0)
x, pv, op = test5(ops)
data[ipv] = x,pv
plot5(x, pv, op, sp, "P:%d"%p)

In [ ]:
sp = 7.1
pv = np.linspace(7.3, 7.1, 10000)
x = np.arange(10000)
plt.gca().clear()
plt.plot(x,  hcalc(sp)-hcalc(pv))
#plt.plot(x, hcalc(sp-pv))

In [ ]:
x = [7+i/10 for i in range(11)]
x = np.array(x)
y = [(hcalc(v-0.2)-hcalc(v)) for v in x]
plt.plot(x,y)

In [ ]:
pidops = ops.co2_pid
pidops.p = 10
pidops.i = 0
pidops.beta = 1
sp = ops.set_point = 7.1
db = ops.set_point_deadband 
ops.initial_pv = 7.3
pv = 7.2096
c = PIDController(pgain=pidops.p,              itime=pidops.i,
                        dtime=pidops.d,             auto_max=pidops.amax,
                        auto_min=pidops.amin,       
                        beta=pidops.beta,
                        linearity=pidops.linearity, alpha=pidops.alpha,
                        deadband=pidops.deadband,   sp_high=100, sp_low=0,
                        gamma=pidops.gamma,         man_request=pidops.man_request,
                        mode=pidops.mode)
co2_sp = hcalc(sp+db)
base_sp = hcalc(sp-db)
h_pv = hcalc(pv)
#c.step(h_pv, co2_sp) / 100

In [ ]:
c.step(h_pv, co2_sp)

In [ ]:
err = co2_sp - h_pv
err * pidops.p

In [ ]:
from types import FunctionType, MethodType
def printdir(o):
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o, a)
            if not isinstance(v, (FunctionType, MethodType)):
                print(a, v)

In [ ]:
printdir(c)

In [ ]:
0 * co2_sp - h_pv